In [1]:
from PIL import Image, ImageFilter
from google.cloud import vision
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import base64
import io
import os

#Funciones generales para uso global
import numpy as np
import time
from shapely.geometry import Polygon, Point
import geopandas as gpd
from shapely.ops import cascaded_union
import rasterio
from rasterio.mask import mask
from shapely import geometry
import pandas as pd
from geotiff import GeoTiff


#Funciones empleadas en la detección de texto
import os, sys
import numpy as np
import cv2
import time
from imutils.object_detection import non_max_suppression

import rasterio
from rasterio.enums import Resampling

# Model v1: Split the image, detect text areas, and convert to text strings

## Splitting the image